In [9]:
import ezc3d
import os
import numpy as np
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt
import tkinter as tk
from tkinter import filedialog
import sqlite3

# ============================ SETUP DB ============================
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ============================ USER INPUT: SELECT DATA FOLDER ============================
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # Extracts 'YYYY-MM-DD' format

# ============================ LOAD & PROCESS C3D FILES ============================
c3d_files = [
    os.path.join(selected_folder, file)
    for file in os.listdir(selected_folder)
    if file.lower().endswith('.c3d')
]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")


def extract_pitch_name(filename):
    """Extracts the pitch name from filenames like 'Slider RH1.c3d' -> 'Slider'."""
    filename_noext = os.path.splitext(filename)[0]  # Remove .c3d
    filename_cleaned = filename_noext.replace("_", " ").replace("-", " ")  # Normalize spaces
    words = filename_cleaned.split()  # Split into words

    if words:
        return words[0].capitalize()  # First word should be the pitch name (Slider, Curve, etc.)
    return "Unknown"


def get_event_time_seconds(c3d, event_name):
    """Returns event time in seconds."""
    if "EVENT" not in c3d["parameters"]:
        raise ValueError(f"No EVENT data in C3D. '{event_name}' not found.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]
    if event_name not in event_labels:
        raise ValueError(f"'{event_name}' event not found in C3D file (event label missing).")
    idx = event_labels.index(event_name)
    return event_times[idx]  # in seconds


def find_local_events(c3d, frame_rate, total_frames):
    """Manual re-offset so earliest event => local frame 0."""
    foot_contact_sec = get_event_time_seconds(c3d, "Foot Contact")
    release_sec      = get_event_time_seconds(c3d, "Release")
    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global      = int(round(release_sec * frame_rate))

    # shift so earliest = local frame 0
    earliest_global = min(foot_contact_global, release_global)
    foot_contact_local = foot_contact_global - earliest_global
    release_local      = release_global - earliest_global

    if foot_contact_local < 0 or foot_contact_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_contact_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")

    return foot_contact_local, release_local


def lowpass_filter(data, cutoff, fs, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=-1)


def compute_segments_over_time(points, marker_indices, start_frame, end_frame):
    forearm_segments = []
    hand_segments = []
    for frame in range(start_frame, end_frame):
        avg_elbow = (points[:3, marker_indices["Lateral_Elbow"], frame] +
                     points[:3, marker_indices["Medial_Elbow"], frame]) / 2
        avg_wrist = (points[:3, marker_indices["Wrist_Radius"], frame] +
                     points[:3, marker_indices["Wrist_Ulna"], frame]) / 2
        hand_pos  = points[:3, marker_indices["Hand"], frame]

        forearm_segments.append((avg_elbow, avg_wrist))
        hand_segments.append((avg_wrist, hand_pos))

    return forearm_segments, hand_segments


def compute_ulnar_deviation(points, marker_indices, frame):
    # Placeholder: real logic would measure angle or offset
    return 0.0


def compute_pronation(points, marker_indices, frame):
    # Placeholder
    return 0.0


def compute_wrist_flexion_pronation(points, marker_indices, start_frame, end_frame):
    """
    Compute wrist flexion and pronation angles over time.
    Returns lists of angles (one per frame).
    """
    flexion_angles = []  # Wrist flexion (frontal, sagittal, transverse)
    pronation_angles = []  # Wrist pronation/supination

    for frame in range(start_frame, end_frame):
        # Get marker positions
        wrist_radius = points[:3, marker_indices["Wrist_Radius"], frame]
        wrist_ulna = points[:3, marker_indices["Wrist_Ulna"], frame]
        forearm = (points[:3, marker_indices["Lateral_Elbow"], frame] +
                   points[:3, marker_indices["Medial_Elbow"], frame]) / 2
        hand = points[:3, marker_indices["Hand"], frame]

        # Compute wrist flexion (angle between hand and forearm)
        forearm_vector = wrist_radius - forearm
        hand_vector = hand - wrist_radius

        # Calculate angle using dot product
        dot_product = np.dot(forearm_vector, hand_vector)
        norm_forearm = np.linalg.norm(forearm_vector)
        norm_hand = np.linalg.norm(hand_vector)
        angle_rad = np.arccos(dot_product / (norm_forearm * norm_hand))
        angle_deg = np.degrees(angle_rad)  # Convert radians to degrees

        flexion_angles.append(angle_deg)

        # Compute pronation/supination (rotation of hand around forearm)
        forearm_axis = wrist_radius - wrist_ulna  # Approximate forearm rotation axis
        hand_movement = hand - wrist_radius  # Hand movement relative to wrist

        cross_product = np.cross(forearm_axis, hand_movement)
        pronation_angle = np.degrees(np.arctan2(np.linalg.norm(cross_product), np.dot(forearm_axis, hand_movement)))

        pronation_angles.append(pronation_angle)

    return flexion_angles, pronation_angles

# We'll store all pitch data for plotting
all_pitches = {}

# We'll process the files & store metrics in SQLite
for c3d_file_path in c3d_files:
    print("Processing:", c3d_file_path)
    c3d = ezc3d.c3d(c3d_file_path)

    # Basic data
    points = c3d["data"]["points"]
    marker_labels = c3d["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    # Attempt to get local events
    try:
        foot_contact_frame, release_frame = find_local_events(c3d, frame_rate, total_frames)
    except ValueError as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # We want release+10
    if release_frame + 10 >= total_frames:
        print(f"Skipping {c3d_file_path}: Not enough frames for release+10.")
        continue

    # Region of interest
    frame_after_20 = min(release_frame + 20, total_frames - 1)
    midway_frame   = (foot_contact_frame + release_frame) // 2

    # Clean marker labels
    marker_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    # define the required markers
    required_markers = ["Lateral_Elbow", "Medial_Elbow","Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    skip_this_file = False
    for mk in required_markers:
        if mk not in marker_labels:
            print(f"Skipping {c3d_file_path}: Marker {mk} not found.")
            skip_this_file = True
            break
        marker_indices[mk] = marker_labels.index(mk)

    if skip_this_file:
        continue
    
    # Extract the pitch name correctly
    filename_only = os.path.basename(c3d_file_path)
    pitch_name = extract_pitch_name(filename_only)
    
    # Ensure pitch_name is initialized in all_pitches before assignment
    if pitch_name not in all_pitches:
        all_pitches[pitch_name] = {
            "forearm": [],
            "hand": [],
            "flexion": [],
            "pronation": []
        }
    
    # Compute wrist flexion and pronation angles
    flexion_angles, pronation_angles = compute_wrist_flexion_pronation(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )
    
    # Store the computed angles
    all_pitches[pitch_name]["flexion"] = flexion_angles
    all_pitches[pitch_name]["pronation"] = pronation_angles
    
    # Compute segments for Dash visualization
    forearm_segments, hand_segments = compute_segments_over_time(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )
    
    all_pitches[pitch_name]["forearm"] = forearm_segments
    all_pitches[pitch_name]["hand"] = hand_segments


    # Filter
    for mk_idx in marker_indices.values():
        points[:3, mk_idx, :] = lowpass_filter(points[:3, mk_idx, :], cutoff=10, fs=frame_rate)

    # Segments for dash
    forearm_segments, hand_segments = compute_segments_over_time(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )

    # ======================================
    # PARSE THE PITCH NAME PROPERLY
    # ======================================
    # original file name e.g. "2025-02-05_\Slider RH 3.c3d"
    filename_only = os.path.basename(c3d_file_path)
    # strip extension => "2025-02-05_\Slider RH 3"
    filename_noext = os.path.splitext(filename_only)[0]
    # split once at underscore => ["2025-02-05", "\Slider RH 3"] or something
    splitted = filename_noext.split("_", 1)
    if len(splitted) == 2:
        # splitted[0] = "2025-02-05"
        # splitted[1] = "\Slider RH 3" or "Slider RH 3"
        # remove any leading slash
        second_chunk = splitted[1].lstrip("\\/")
        # Now get the first word => "Slider"
        pitch_type_str = second_chunk.split()[0].lower()  # "slider"
        pitch_type_str = pitch_type_str.replace("rh", "") # if you want to remove "RH" from "Slider RH"
        pitch_type_str = pitch_type_str.capitalize()      # => "Slider"
    else:
        pitch_type_str = "Unknown"

    # If it is a static
    if "static" in pitch_type_str.lower():
        print(f"Skipping static: {c3d_file_path}")
        continue

    # Now we have a pitch name like "Slider" or "Curve" or "Fastball" etc.
    # Store it in all_pitches for dash
    if pitch_type_str not in all_pitches:
        all_pitches[pitch_type_str] = {"forearm": [], "hand": []}
    # We can just store the last one or append. Typically we just store one set,
    # but if you want multiple sets, you might store them in a list.
    all_pitches[pitch_type_str]["forearm"].extend(forearm_segments)
    all_pitches[pitch_type_str]["hand"].extend(hand_segments)

    # ======================================
    # SAVE METRICS TO SQLite
    # ======================================
    # subject folder = -2
    path_parts = c3d_file_path.split(os.sep)
    subject_folder = path_parts[-2].replace("_BW", "")  # e.g. "Bobby Wahl"
    # The last part is the filename
    # But we also want the chunk "2025-02-05_" maybe from splitted[0]
    date_folder = splitted[0] + "_"  # => "2025-02-05_"

    # create table if not exist
    table_name = pitch_type_str  # e.g. "Slider", "Curve", etc.

    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            subject_folder TEXT,
            date_folder TEXT,
            filename TEXT,
            mid_u_dev REAL,
            rel_u_dev REAL,
            frame1_u_dev REAL,
            frame2_u_dev REAL,
            frame3_u_dev REAL,
            frame4_u_dev REAL,
            frame5_u_dev REAL,
            frame6_u_dev REAL,
            frame7_u_dev REAL,
            frame8_u_dev REAL,
            frame9_u_dev REAL,
            frame10_u_dev REAL,
            mid_pronation REAL,
            rel_pronation REAL,
            frame1_pronation REAL,
            frame2_pronation REAL,
            frame3_pronation REAL,
            frame4_pronation REAL,
            frame5_pronation REAL,
            frame6_pronation REAL,
            frame7_pronation REAL,
            frame8_pronation REAL,
            frame9_pronation REAL,
            frame10_pronation REAL
        )
    """)

    # compute angles
    mid_u_dev = compute_ulnar_deviation(points, marker_indices, midway_frame)
    rel_u_dev = compute_ulnar_deviation(points, marker_indices, release_frame)
    frames_u_dev = [
        compute_ulnar_deviation(points, marker_indices, release_frame + i)
        for i in range(1, 11)
    ]

    mid_pro = compute_pronation(points, marker_indices, midway_frame)
    rel_pro = compute_pronation(points, marker_indices, release_frame)
    frames_pro = [
        compute_pronation(points, marker_indices, release_frame + i)
        for i in range(1, 11)
    ]

    insert_sql = f"""
        INSERT INTO {table_name} (
            subject_folder, date_folder, filename,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation,
            frame5_pronation, frame6_pronation, frame7_pronation, frame8_pronation,
            frame9_pronation, frame10_pronation
        ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        subject_folder,                # subject_folder
        date_folder,                   # date_folder
        filename_only,                 # full filename e.g. "2025-02-05_\Slider RH 3.c3d"
        mid_u_dev,                     # mid_u_dev
        rel_u_dev,                     # rel_u_dev
        frames_u_dev[0], frames_u_dev[1], frames_u_dev[2], frames_u_dev[3], frames_u_dev[4],
        frames_u_dev[5], frames_u_dev[6], frames_u_dev[7], frames_u_dev[8], frames_u_dev[9],
        mid_pro,                       # mid_pronation
        rel_pro,                       # rel_pronation
        frames_pro[0], frames_pro[1], frames_pro[2], frames_pro[3],
        frames_pro[4], frames_pro[5], frames_pro[6], frames_pro[7],
        frames_pro[8], frames_pro[9]
    )
    cursor.execute(insert_sql, data_tuple)

# Finally, commit and close
conn.commit()
conn.close()

# ============================ DASH APP ============================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

pitch_color_map = {
    "Curve": "#32CD32", #lime green
    "Fastball": "#FF0000",  # Red
    "Changeup": "green",
    "Slider": "yellow",
    "Cutter": "#FF0000"  # Red
}

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("3D Visualization of Forearm & Hand Over Time"), className="text-center")
    ]),

    # Dropdown for pitch selection
    dbc.Row([
        dbc.Col(html.Label("Select Pitches to Overlay:"), width=4),
        dbc.Col(dcc.Dropdown(
            id="pitch-selector",
            options=[{"label": p, "value": p} for p in all_pitches.keys()],
            value=[],
            multi=True
        ), width=8)
    ]),

    # Radio button for view selection
    dbc.Row([
        dbc.Col(html.Label("Select View:"), width=4),
        dbc.Col(dcc.RadioItems(
            id="view-toggle",
            options=[
                {"label": "Rear View (XY)", "value": "rear"},
                {"label": "Side View (XZ)", "value": "side"}
            ],
            value="rear",
            inline=True
        ), width=8)
    ]),

    # 3D Graph (Top Middle)
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-3d"), width=12)
    ]),

    # 2D Graphs for forearm-hand segments
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-2d-1"), width=6),
        dbc.Col(dcc.Graph(id="pitch-graph-2d-2"), width=6)
    ]),

    # Radial/Ulnar Deviation (Centered Above Other Wrist Graphs)
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-ulnar"), width=12)
    ]),

    # Bottom Row: Pronation (Left) & Flexion/Extension (Right)
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-pronation"), width=6),
        dbc.Col(dcc.Graph(id="pitch-graph-flexion"), width=6)
    ]),
    
    # Add frame selection controls
    dbc.Row([
        dbc.Col(html.Label("Select Frame Range:"), width=4),
        dbc.Col(dcc.RangeSlider(
            id="frame-slider",
            min=0,
            max=100,  # This will be dynamically updated
            step=1,
            value=[0, 100],  # Default: Show all frames
            marks={0: "0", 100: "100"}
        ), width=6),
        dbc.Col(dbc.Button("Show All Frames", id="reset-slider", color="primary"), width=2)
    ])

])



def create_3d_figure(selected_pitches, view="rear", selected_frame=0):
    fig = go.Figure()

    for pitch_name in selected_pitches:
        if pitch_name not in all_pitches:
            continue

        color = pitch_color_map.get(pitch_name, "gray")
        forearm = all_pitches[pitch_name]["forearm"]
        hand = all_pitches[pitch_name]["hand"]

        if selected_frame < len(forearm):  # Ensure frame is within range
            # Forearm segment
            fig.add_trace(go.Scatter3d(
                x=[forearm[selected_frame][0][0], forearm[selected_frame][1][0]],
                y=[forearm[selected_frame][0][1], forearm[selected_frame][1][1]] if view == "rear" else [forearm[selected_frame][0][2], forearm[selected_frame][1][2]],
                z=[forearm[selected_frame][0][2], forearm[selected_frame][1][2]] if view == "rear" else [forearm[selected_frame][0][1], forearm[selected_frame][1][1]],
                mode='lines',
                line=dict(width=5, color=color),
                name=f"{pitch_name} Forearm"
            ))

            # Hand segment
            fig.add_trace(go.Scatter3d(
                x=[hand[selected_frame][0][0], hand[selected_frame][1][0]],
                y=[hand[selected_frame][0][1], hand[selected_frame][1][1]] if view == "rear" else [hand[selected_frame][0][2], hand[selected_frame][1][2]],
                z=[hand[selected_frame][0][2], hand[selected_frame][1][2]] if view == "rear" else [hand[selected_frame][0][1], hand[selected_frame][1][1]],
                mode='lines',
                line=dict(width=5, color=color),
                name=f"{pitch_name} Hand"
            ))

    fig.update_layout(
        title=f"Frame {selected_frame} - Forearm & Hand",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y" if view == "rear" else "Z",
            zaxis_title="Z" if view == "rear" else "Y"
        )
    )
    return fig



def create_2d_figure(selected_pitches, x_axis="x", y_axis="z"):
    axis_index_map = {"x": 0, "y": 1, "z": 2}
    ix = axis_index_map[x_axis]
    iy = axis_index_map[y_axis]

    fig = go.Figure()
    for pitch_name in selected_pitches:
        if pitch_name not in all_pitches:
            continue
        color = pitch_color_map.get(pitch_name, "gray")
        forearm = all_pitches[pitch_name]["forearm"]
        hand    = all_pitches[pitch_name]["hand"]
        for i in range(len(forearm)):
            fig.add_trace(go.Scatter(
                x=[forearm[i][0][ix], forearm[i][1][ix]],
                y=[forearm[i][0][iy], forearm[i][1][iy]],
                mode='lines',
                line=dict(width=2, color=color),
                name=f"{pitch_name} Forearm",
                showlegend=(i == 0)  # Only show legend once
            ))
            fig.add_trace(go.Scatter(
                x=[hand[i][0][ix], hand[i][1][ix]],
                y=[hand[i][0][iy], hand[i][1][iy]],
                mode='lines',
                line=dict(width=2, color=color),
                name=f"{pitch_name} Hand",
                showlegend=(i == 0)
            ))
    fig.update_layout(
        title=f"2D View: {y_axis.upper()} vs. {x_axis.upper()}",
        xaxis_title=x_axis.upper(),
        yaxis_title=y_axis.upper(),
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )
    return fig


def create_ulnar_deviation_graph(selected_pitches, selected_frames=None):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue

        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["flexion"])))  # X-axis (frames)
        ulnar_dev = all_pitches[pitch]["flexion"]  # Y-axis (deviation)

        # Apply frame selection if enabled
        if selected_frames is not None:
            frames = frames[selected_frames[0]:selected_frames[1]]
            ulnar_dev = ulnar_dev[selected_frames[0]:selected_frames[1]]

        fig.add_trace(go.Scatter(
            x=frames, y=ulnar_dev,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Ulnar Deviation"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder, update with real release frame
        fig.add_vline(x=release_frame, line=dict(color="black", width=2, dash="dash"), name="Release")

    fig.update_layout(
        title="Ulnar/Radial Deviation Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig




def create_pronation_graph(selected_pitches, selected_frames=None):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue
        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["pronation"])))
        pronation = all_pitches[pitch]["pronation"]

        # Apply frame selection if enabled
        if selected_frames is not None:
            frames = frames[selected_frames[0]:selected_frames[1]]
            pronation = pronation[selected_frames[0]:selected_frames[1]]

        fig.add_trace(go.Scatter(
            x=frames, y=pronation,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Pronation"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder, update with real release frame
        fig.add_vline(x=release_frame, line=dict(color="black", width=2, dash="dash"), name="Release")

    fig.update_layout(
        title="Pronation/Supination Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig




def create_flexion_graph(selected_pitches, selected_frames=None):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue
        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["flexion"])))
        flexion = all_pitches[pitch]["flexion"]

        # Apply frame selection if enabled
        if selected_frames is not None:
            frames = frames[selected_frames[0]:selected_frames[1]]
            flexion = flexion[selected_frames[0]:selected_frames[1]]

        fig.add_trace(go.Scatter(
            x=frames, y=flexion,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Flexion"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder, update with real release frame
        fig.add_vline(x=release_frame, line=dict(color="black", width=2, dash="dash"), name="Release")

    fig.update_layout(
        title="Flexion/Extension Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig




from dash.dependencies import State
import time

from dash.dependencies import State

@app.callback(
    [Output("pitch-graph-3d", "figure"),
     Output("pitch-graph-2d-1", "figure"),
     Output("pitch-graph-2d-2", "figure"),
     Output("pitch-graph-ulnar", "figure"),
     Output("pitch-graph-pronation", "figure"),
     Output("pitch-graph-flexion", "figure"),
     Output("frame-slider", "max"),
     Output("frame-slider", "marks"),
     Output("frame-slider", "value")],  # Update max and marks dynamically
    [Input("pitch-selector", "value"),
     Input("view-toggle", "value"),
     Input("frame-slider", "value"),
     Input("reset-slider", "n_clicks")]
)
def update_all_graphs(selected_pitches, view, selected_frames, reset_click):
    # Get the max number of frames dynamically
    max_frames = 100
    for pitch in selected_pitches:
        if pitch in all_pitches:
            max_frames = max(max_frames, len(all_pitches[pitch]["flexion"]))

    # Update slider marks
    marks = {0: "0", max_frames: str(max_frames)}

    # If reset button is clicked, reset slider to full range
    if reset_click:
        selected_frames = [0, max_frames]

    # Generate updated graphs with selected frame range
    fig_3d = create_3d_figure(selected_pitches, view)
    fig_zx_2d = create_2d_figure(selected_pitches, x_axis="x", y_axis="z")
    fig_zy_2d = create_2d_figure(selected_pitches, x_axis="y", y_axis="z")
    fig_ulnar = create_ulnar_deviation_graph(selected_pitches, selected_frames)
    fig_pronation = create_pronation_graph(selected_pitches, selected_frames)
    fig_flexion = create_flexion_graph(selected_pitches, selected_frames)

    return fig_3d, fig_zx_2d, fig_zy_2d, fig_ulnar, fig_pronation, fig_flexion, max_frames, marks, selected_frames




if __name__ == '__main__':
    app.run_server(debug=True)
    print('Visit: http://127.0.0.1:8050/')


Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 2.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Changeup RH 2.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Changeup RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 6.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 5.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 4.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Fastball RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 5.c3d
Processing: D:/Youth Pitch 

http://127.0.0.1:8050/
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 3.c3d


OperationalError: near "rh": syntax error

In [23]:
import ezc3d
import os
import numpy as np
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt
import tkinter as tk
from tkinter import filedialog
import sqlite3

# ============================ SETUP DB ============================
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ============================ USER INPUT: SELECT DATA FOLDER ============================
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # Extracts 'YYYY-MM-DD' format

# ============================ LOAD & PROCESS C3D FILES ============================
c3d_files = [
    os.path.join(selected_folder, file)
    for file in os.listdir(selected_folder)
    if file.lower().endswith('.c3d')
]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")


def extract_pitch_name(filename):
    """Extracts the pitch name from filenames like 'Slider RH1.c3d' -> 'Slider'."""
    filename_noext = os.path.splitext(filename)[0]  # Remove .c3d
    filename_cleaned = filename_noext.replace("_", " ").replace("-", " ")  # Normalize spaces
    words = filename_cleaned.split()  # Split into words

    if words:
        return words[0].capitalize()  # First word should be the pitch name (Slider, Curve, etc.)
    return "Unknown"


def get_event_time_seconds(c3d, event_name):
    """Returns event time in seconds."""
    if "EVENT" not in c3d["parameters"]:
        raise ValueError(f"No EVENT data in C3D. '{event_name}' not found.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]
    if event_name not in event_labels:
        raise ValueError(f"'{event_name}' event not found in C3D file (event label missing).")
    idx = event_labels.index(event_name)
    return event_times[idx]  # in seconds


def find_local_events(c3d, frame_rate, total_frames):
    """Manual re-offset so earliest event => local frame 0."""
    foot_contact_sec = get_event_time_seconds(c3d, "Foot Contact")
    release_sec      = get_event_time_seconds(c3d, "Release")
    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global      = int(round(release_sec * frame_rate))

    # shift so earliest = local frame 0
    earliest_global = min(foot_contact_global, release_global)
    foot_contact_local = foot_contact_global - earliest_global
    release_local      = release_global - earliest_global

    if foot_contact_local < 0 or foot_contact_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_contact_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")

    return foot_contact_local, release_local


def lowpass_filter(data, cutoff, fs, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=-1)


def compute_segments_over_time(points, marker_indices, start_frame, end_frame):
    forearm_segments = []
    hand_segments = []
    for frame in range(start_frame, end_frame):
        avg_elbow = (points[:3, marker_indices["Lateral_Elbow"], frame] +
                     points[:3, marker_indices["Medial_Elbow"], frame]) / 2
        avg_wrist = (points[:3, marker_indices["Wrist_Radius"], frame] +
                     points[:3, marker_indices["Wrist_Ulna"], frame]) / 2
        hand_pos  = points[:3, marker_indices["Hand"], frame]

        forearm_segments.append((avg_elbow, avg_wrist))
        hand_segments.append((avg_wrist, hand_pos))

    return forearm_segments, hand_segments


def compute_ulnar_deviation(points, marker_indices, frame):
    # Placeholder: real logic would measure angle or offset
    return 0.0


def compute_pronation(points, marker_indices, frame):
    # Placeholder
    return 0.0


def compute_wrist_flexion_pronation(points, marker_indices, start_frame, end_frame):
    """
    Compute wrist flexion and pronation angles over time.
    Returns lists of angles (one per frame).
    """
    flexion_angles = []  # Wrist flexion (frontal, sagittal, transverse)
    pronation_angles = []  # Wrist pronation/supination

    for frame in range(start_frame, end_frame):
        # Get marker positions
        wrist_radius = points[:3, marker_indices["Wrist_Radius"], frame]
        wrist_ulna = points[:3, marker_indices["Wrist_Ulna"], frame]
        forearm = (points[:3, marker_indices["Lateral_Elbow"], frame] +
                   points[:3, marker_indices["Medial_Elbow"], frame]) / 2
        hand = points[:3, marker_indices["Hand"], frame]

        # Compute wrist flexion (angle between hand and forearm)
        forearm_vector = wrist_radius - forearm
        hand_vector = hand - wrist_radius

        # Calculate angle using dot product
        dot_product = np.dot(forearm_vector, hand_vector)
        norm_forearm = np.linalg.norm(forearm_vector)
        norm_hand = np.linalg.norm(hand_vector)
        angle_rad = np.arccos(dot_product / (norm_forearm * norm_hand))
        angle_deg = np.degrees(angle_rad)  # Convert radians to degrees

        flexion_angles.append(angle_deg)

        # Compute pronation/supination (rotation of hand around forearm)
        forearm_axis = wrist_radius - wrist_ulna  # Approximate forearm rotation axis
        hand_movement = hand - wrist_radius  # Hand movement relative to wrist

        cross_product = np.cross(forearm_axis, hand_movement)
        pronation_angle = np.degrees(np.arctan2(np.linalg.norm(cross_product), np.dot(forearm_axis, hand_movement)))

        pronation_angles.append(pronation_angle)

    return flexion_angles, pronation_angles

# We'll store all pitch data for plotting
all_pitches = {}

# We'll process the files & store metrics in SQLite
for c3d_file_path in c3d_files:
    print("Processing:", c3d_file_path)
    c3d = ezc3d.c3d(c3d_file_path)

    # Basic data
    points = c3d["data"]["points"]
    marker_labels = c3d["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    # Attempt to get local events
    try:
        foot_contact_frame, release_frame = find_local_events(c3d, frame_rate, total_frames)
    except ValueError as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # We want release+10
    if release_frame + 10 >= total_frames:
        print(f"Skipping {c3d_file_path}: Not enough frames for release+10.")
        continue

    # Region of interest
    frame_after_20 = min(release_frame + 20, total_frames - 1)
    midway_frame   = (foot_contact_frame + release_frame) // 2

    # Clean marker labels
    marker_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    # define the required markers
    required_markers = ["Lateral_Elbow", "Medial_Elbow","Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    skip_this_file = False
    for mk in required_markers:
        if mk not in marker_labels:
            print(f"Skipping {c3d_file_path}: Marker {mk} not found.")
            skip_this_file = True
            break
        marker_indices[mk] = marker_labels.index(mk)

    if skip_this_file:
        continue
    
    # Extract the pitch name correctly
    filename_only = os.path.basename(c3d_file_path)
    pitch_name = extract_pitch_name(filename_only)
    
    # Ensure pitch_name is initialized in all_pitches before assignment
    if pitch_name not in all_pitches:
        all_pitches[pitch_name] = {
            "forearm": [],
            "hand": [],
            "flexion": [],
            "pronation": []
        }
    
    # Compute wrist flexion and pronation angles
    flexion_angles, pronation_angles = compute_wrist_flexion_pronation(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )
    
    # Store the computed angles
    all_pitches[pitch_name]["flexion"] = flexion_angles
    all_pitches[pitch_name]["pronation"] = pronation_angles
    
    # Compute segments for Dash visualization
    forearm_segments, hand_segments = compute_segments_over_time(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )
    
    all_pitches[pitch_name]["forearm"] = forearm_segments
    all_pitches[pitch_name]["hand"] = hand_segments


    # Filter
    for mk_idx in marker_indices.values():
        points[:3, mk_idx, :] = lowpass_filter(points[:3, mk_idx, :], cutoff=10, fs=frame_rate)

    # Segments for dash
    forearm_segments, hand_segments = compute_segments_over_time(
        points, marker_indices, start_frame=midway_frame, end_frame=frame_after_20
    )

    # ======================================
    # PARSE THE PITCH NAME PROPERLY
    # ======================================
    # original file name e.g. "2025-02-05_\Slider RH 3.c3d"
    filename_only = os.path.basename(c3d_file_path)
    # strip extension => "2025-02-05_\Slider RH 3"
    filename_noext = os.path.splitext(filename_only)[0]
    # split once at underscore => ["2025-02-05", "\Slider RH 3"] or something
    splitted = filename_noext.split("_", 1)
    if len(splitted) == 2:
        # splitted[0] = "2025-02-05"
        # splitted[1] = "\Slider RH 3" or "Slider RH 3"
        # remove any leading slash
        second_chunk = splitted[1].lstrip("\\/")
        # Now get the first word => "Slider"
        pitch_type_str = second_chunk.split()[0].lower()  # "slider"
        pitch_type_str = pitch_type_str.replace("rh", "") # if you want to remove "RH" from "Slider RH"
        pitch_type_str = pitch_type_str.capitalize()      # => "Slider"
    else:
        pitch_type_str = "Unknown"

    # If it is a static
    if "static" in pitch_type_str.lower():
        print(f"Skipping static: {c3d_file_path}")
        continue

    # Now we have a pitch name like "Slider" or "Curve" or "Fastball" etc.
    # Store it in all_pitches for dash
    if pitch_type_str not in all_pitches:
        all_pitches[pitch_type_str] = {"forearm": [], "hand": []}
    # We can just store the last one or append. Typically we just store one set,
    # but if you want multiple sets, you might store them in a list.
    all_pitches[pitch_type_str]["forearm"].extend(forearm_segments)
    all_pitches[pitch_type_str]["hand"].extend(hand_segments)

    # ======================================
    # SAVE METRICS TO SQLite
    # ======================================
    # subject folder = -2
    path_parts = c3d_file_path.split(os.sep)
    subject_folder = path_parts[-2].replace("_BW", "")  # e.g. "Bobby Wahl"
    # The last part is the filename
    # But we also want the chunk "2025-02-05_" maybe from splitted[0]
    date_folder = splitted[0] + "_"  # => "2025-02-05_"

    # create table if not exist
    table_name = pitch_type_str  # e.g. "Slider", "Curve", etc.

    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            subject_folder TEXT,
            date_folder TEXT,
            filename TEXT,
            mid_u_dev REAL,
            rel_u_dev REAL,
            frame1_u_dev REAL,
            frame2_u_dev REAL,
            frame3_u_dev REAL,
            frame4_u_dev REAL,
            frame5_u_dev REAL,
            frame6_u_dev REAL,
            frame7_u_dev REAL,
            frame8_u_dev REAL,
            frame9_u_dev REAL,
            frame10_u_dev REAL,
            mid_pronation REAL,
            rel_pronation REAL,
            frame1_pronation REAL,
            frame2_pronation REAL,
            frame3_pronation REAL,
            frame4_pronation REAL,
            frame5_pronation REAL,
            frame6_pronation REAL,
            frame7_pronation REAL,
            frame8_pronation REAL,
            frame9_pronation REAL,
            frame10_pronation REAL
        )
    """)

    # compute angles
    mid_u_dev = compute_ulnar_deviation(points, marker_indices, midway_frame)
    rel_u_dev = compute_ulnar_deviation(points, marker_indices, release_frame)
    frames_u_dev = [
        compute_ulnar_deviation(points, marker_indices, release_frame + i)
        for i in range(1, 11)
    ]

    mid_pro = compute_pronation(points, marker_indices, midway_frame)
    rel_pro = compute_pronation(points, marker_indices, release_frame)
    frames_pro = [
        compute_pronation(points, marker_indices, release_frame + i)
        for i in range(1, 11)
    ]

    insert_sql = f"""
        INSERT INTO {table_name} (
            subject_folder, date_folder, filename,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation,
            frame5_pronation, frame6_pronation, frame7_pronation, frame8_pronation,
            frame9_pronation, frame10_pronation
        ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        subject_folder,                # subject_folder
        date_folder,                   # date_folder
        filename_only,                 # full filename e.g. "2025-02-05_\Slider RH 3.c3d"
        mid_u_dev,                     # mid_u_dev
        rel_u_dev,                     # rel_u_dev
        frames_u_dev[0], frames_u_dev[1], frames_u_dev[2], frames_u_dev[3], frames_u_dev[4],
        frames_u_dev[5], frames_u_dev[6], frames_u_dev[7], frames_u_dev[8], frames_u_dev[9],
        mid_pro,                       # mid_pronation
        rel_pro,                       # rel_pronation
        frames_pro[0], frames_pro[1], frames_pro[2], frames_pro[3],
        frames_pro[4], frames_pro[5], frames_pro[6], frames_pro[7],
        frames_pro[8], frames_pro[9]
    )
    cursor.execute(insert_sql, data_tuple)

# Finally, commit and close
conn.commit()
conn.close()

# ============================ DASH APP ============================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

pitch_color_map = {
    "Curve": "#32CD32", #lime green
    "Fastball": "#FF0000",  # Red
    "Changeup": "green",
    "Slider": "yellow",
    "Cutter": "#FF0000"  # Red
}

app.layout = dbc.Container([
    # Title
    dbc.Row([
        dbc.Col(html.H2("3D Visualization of Forearm & Hand Over Time"), className="text-center")
    ]),

    # Dropdown for pitch selection
    dbc.Row([
        dbc.Col(html.Label("Select Pitches to Overlay:"), width=4),
        dbc.Col(dcc.Dropdown(
            id="pitch-selector",
            options=[{"label": p, "value": p} for p in all_pitches.keys()],
            value=[],
            multi=True
        ), width=8)
    ]),

    # View selection added back
    dbc.Row([
        dbc.Col(html.Label("Select View:"), width=4),
        dbc.Col(dcc.RadioItems(
            id="view-toggle",
            options=[
                {"label": "Rear View (XY)", "value": "rear"},
                {"label": "Side View (XZ)", "value": "side"}
            ],
            value="rear",
            inline=True
        ), width=8)
    ]),

    # View selection & frame scrubber positioned **next to the 3D Graph**
    dbc.Row([
        # Left Side - 3D Graph
        dbc.Col(dcc.Graph(id="pitch-graph-3d"), width=10),

        # Right Side - Scrubber & Play Button
        dbc.Col([
            html.Label("Scrub Through Frames:"),
            dcc.Slider(
                id="frame-slider",
                min=0,
                max=100,  # Will be dynamically updated
                step=1,
                value=0,  # Default: Show frame 0
                marks={0: "0", 100: "100"}
            ),
            html.Br(),
            dbc.Button("Play", id="play-button", color="success"),
            
            # Auto-play interval component
            dcc.Interval(
                id="frame-interval",
                interval=200,  # Updates every 200ms (adjust for speed)
                n_intervals=0,
                disabled=True  # Start in disabled state
            ),
        ], width=2)
    ]),

    # 2D Graphs
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-2d-1"), width=6),
        dbc.Col(dcc.Graph(id="pitch-graph-2d-2"), width=6)
    ]),

    # Ulnar Deviation (Above Pronation/Flexion)
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-ulnar"), width=12)
    ]),

    # Bottom Row: Pronation & Flexion
    dbc.Row([
        dbc.Col(dcc.Graph(id="pitch-graph-pronation"), width=6),
        dbc.Col(dcc.Graph(id="pitch-graph-flexion"), width=6)
    ]),

])


def create_3d_figure(selected_pitches, view="rear", selected_frame=0):
    fig = go.Figure()

    for pitch_name in selected_pitches:
        if pitch_name not in all_pitches:
            continue

        color = pitch_color_map.get(pitch_name, "gray")
        forearm = all_pitches[pitch_name]["forearm"]
        hand = all_pitches[pitch_name]["hand"]

        if selected_frame < len(forearm):  # Ensure frame is within range
            # Forearm segment
            fig.add_trace(go.Scatter3d(
                x=[forearm[selected_frame][0][0], forearm[selected_frame][1][0]],
                y=[forearm[selected_frame][0][1], forearm[selected_frame][1][1]] if view == "rear" else [forearm[selected_frame][0][2], forearm[selected_frame][1][2]],
                z=[forearm[selected_frame][0][2], forearm[selected_frame][1][2]] if view == "rear" else [forearm[selected_frame][0][1], forearm[selected_frame][1][1]],
                mode='lines',
                line=dict(width=5, color=color),
                name=f"{pitch_name} Forearm"
            ))

            # Hand segment
            fig.add_trace(go.Scatter3d(
                x=[hand[selected_frame][0][0], hand[selected_frame][1][0]],
                y=[hand[selected_frame][0][1], hand[selected_frame][1][1]] if view == "rear" else [hand[selected_frame][0][2], hand[selected_frame][1][2]],
                z=[hand[selected_frame][0][2], hand[selected_frame][1][2]] if view == "rear" else [hand[selected_frame][0][1], hand[selected_frame][1][1]],
                mode='lines',
                line=dict(width=5, color=color),
                name=f"{pitch_name} Hand"
            ))

    fig.update_layout(
        title=f"Frame {selected_frame} - Forearm & Hand",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y" if view == "rear" else "Z",
            zaxis_title="Z" if view == "rear" else "Y"
        )
    )
    return fig



def create_2d_figure(selected_pitches, x_axis="x", y_axis="z"):
    axis_index_map = {"x": 0, "y": 1, "z": 2}
    ix = axis_index_map[x_axis]
    iy = axis_index_map[y_axis]

    fig = go.Figure()
    for pitch_name in selected_pitches:
        if pitch_name not in all_pitches:
            continue
        color = pitch_color_map.get(pitch_name, "gray")
        forearm = all_pitches[pitch_name]["forearm"]
        hand    = all_pitches[pitch_name]["hand"]
        for i in range(len(forearm)):
            fig.add_trace(go.Scatter(
                x=[forearm[i][0][ix], forearm[i][1][ix]],
                y=[forearm[i][0][iy], forearm[i][1][iy]],
                mode='lines',
                line=dict(width=2, color=color),
                name=f"{pitch_name} Forearm",
                showlegend=(i == 0)  # Only show legend once
            ))
            fig.add_trace(go.Scatter(
                x=[hand[i][0][ix], hand[i][1][ix]],
                y=[hand[i][0][iy], hand[i][1][iy]],
                mode='lines',
                line=dict(width=2, color=color),
                name=f"{pitch_name} Hand",
                showlegend=(i == 0)
            ))
    fig.update_layout(
        title=f"2D View: {y_axis.upper()} vs. {x_axis.upper()}",
        xaxis_title=x_axis.upper(),
        yaxis_title=y_axis.upper(),
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )
    return fig


def create_ulnar_deviation_graph(selected_pitches, selected_frame=0):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue
        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["flexion"])))  
        ulnar_dev = all_pitches[pitch]["flexion"]

        # Ensure selected_frame is within valid range
        selected_frame = min(max(selected_frame, 0), len(frames) - 1)

        fig.add_trace(go.Scatter(
            x=frames, y=ulnar_dev,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Ulnar Deviation"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder for actual release frame
        fig.add_vline(x=release_frame, line=dict(color="white", width=2, dash="dash"), name="Release")

        # Add moving vertical line for scrubbed frame
        fig.add_vline(x=selected_frame, line=dict(color="black", width=2, dash="dash"), name="Current Frame")

    fig.update_layout(
        title="Ulnar/Radial Deviation Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig



def create_pronation_graph(selected_pitches, selected_frame=0):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue
        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["pronation"])))
        pronation = all_pitches[pitch]["pronation"]

        # Ensure selected_frame is within valid range
        selected_frame = min(max(selected_frame, 0), len(frames) - 1)

        fig.add_trace(go.Scatter(
            x=frames, y=pronation,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Pronation"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder for actual release frame
        fig.add_vline(x=release_frame, line=dict(color="white", width=2, dash="dash"), name="Release")

        # Add moving vertical line for scrubbed frame
        fig.add_vline(x=selected_frame, line=dict(color="black", width=2, dash="dash"), name="Current Frame")

    fig.update_layout(
        title="Pronation/Supination Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig



def create_flexion_graph(selected_pitches, selected_frame=0):
    fig = go.Figure()
    for pitch in selected_pitches:
        if pitch not in all_pitches:
            continue
        color = pitch_color_map.get(pitch, "black")
        frames = list(range(len(all_pitches[pitch]["flexion"])))
        flexion = all_pitches[pitch]["flexion"]

        # Ensure selected_frame is within valid range
        selected_frame = min(max(selected_frame, 0), len(frames) - 1)

        fig.add_trace(go.Scatter(
            x=frames, y=flexion,
            mode='lines', line=dict(width=2, color=color),
            name=f"{pitch} Flexion"
        ))

        # Add vertical line for Release
        release_frame = len(frames) // 2  # Placeholder for actual release frame
        fig.add_vline(x=release_frame, line=dict(color="white", width=2, dash="dash"), name="Release")

        # Add moving vertical line for scrubbed frame
        fig.add_vline(x=selected_frame, line=dict(color="black", width=2, dash="dash"), name="Current Frame")

    fig.update_layout(
        title="Flexion/Extension Over Time",
        xaxis_title="Frame",
        yaxis_title="Angle (degrees)"
    )
    return fig



from dash.dependencies import State
import time

from dash.dependencies import State

@app.callback(
    [Output("pitch-graph-3d", "figure"),
     Output("pitch-graph-2d-1", "figure"),
     Output("pitch-graph-2d-2", "figure"),
     Output("pitch-graph-ulnar", "figure"),
     Output("pitch-graph-pronation", "figure"),
     Output("pitch-graph-flexion", "figure"),
     Output("frame-slider", "max"),
     Output("frame-slider", "marks"),
     Output("frame-slider", "value"),
     Output("frame-interval", "disabled")],  # Control Interval Component
    [Input("pitch-selector", "value"),
     Input("view-toggle", "value"),
     Input("frame-slider", "value"),
     Input("play-button", "n_clicks"),
     Input("frame-interval", "n_intervals")],  # Tracks automatic frame updates
    [State("frame-slider", "max")]
)
def update_all_graphs(selected_pitches, view, selected_frame, play_click, n_intervals, max_frames):
    ctx = dash.callback_context
    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0] if ctx.triggered else None

    # If Play button is clicked, enable auto-update
    play_mode = triggered_id == "play-button" or (triggered_id == "frame-interval" and n_intervals > 0)

    # When playing, move to the next frame
    if play_mode:
        selected_frame = (selected_frame + 1) % (max_frames + 1)  # Loop back after last frame

    # Graph Updates
    fig_3d = create_3d_figure(selected_pitches, view, selected_frame)
    fig_zx_2d = create_2d_figure(selected_pitches, x_axis="x", y_axis="z")
    fig_zy_2d = create_2d_figure(selected_pitches, x_axis="y", y_axis="z")
    fig_ulnar = create_ulnar_deviation_graph(selected_pitches, selected_frame)
    fig_pronation = create_pronation_graph(selected_pitches, selected_frame)
    fig_flexion = create_flexion_graph(selected_pitches, selected_frame)

    return (fig_3d, fig_zx_2d, fig_zy_2d, fig_ulnar, fig_pronation, fig_flexion,
            max_frames, {0: "0", max_frames: str(max_frames)}, selected_frame,
            not play_mode)  # Disable interval when not playing




if __name__ == '__main__':
    app.run_server(debug=True)
    print('Visit: http://127.0.0.1:8050/')


Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 2.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Changeup RH 2.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Changeup RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 6.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 5.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 4.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Curve RH 1.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Fastball RH 3.c3d
Processing: D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Slider RH 5.c3d
Processing: D:/Youth Pitch 

Visit: http://127.0.0.1:8050/
